In [3]:
from nnet_for_hist_dropout_stride import *
import logging

In [2]:
if __name__ == "__main__":
    predict_year = 2016
    logging.basicConfig(filename='train_for_hist_alldata'+str(predict_year)+'.log',level=logging.DEBUG)
    # Create a coordinator
    config = Config()

    # load data to memory
    filename = 'histogram_all_full' + '.npz'
    print(config.load_path)
    # filename = 'histogram_all_soilweather' + '.npz'
    content = np.load(config.load_path + filename)
    image_all = content['output_image']
    yield_all = content['output_yield']
    year_all = content['output_year']
    print(year_all)
    locations_all = content['output_locations']
    index_all = content['output_index']

     # delete broken image
    list_delete=[]
    for i in range(image_all.shape[0]):
        if np.sum(image_all[i,:,:,:])<=223:
            if year_all[i]<2016:
                list_delete.append(i)
    image_all=np.delete(image_all,list_delete,0)
    yield_all=np.delete(yield_all,list_delete,0)
    year_all = np.delete(year_all,list_delete, 0)
    print(year_all)
    locations_all = np.delete(locations_all, list_delete, 0)
    index_all = np.delete(index_all, list_delete, 0)


    # keep major counties
    #list_keep=[]
    #for i in range(image_all.shape[0]):
    #   if (index_all[i,0]==5)or(index_all[i,0]==17)or(index_all[i,0]==18)or(index_all[i,0]==19)or(index_all[i,0]==20)or(index_all[i,0]==27)or(index_all[i,0]==29)or(index_all[i,0]==31)or(index_all[i,0]==38)or(index_all[i,0]==39)or(index_all[i,0]==46):
    #        list_keep.append(i)
    #image_all=image_all[list_keep,:,:,:]
    #yield_all=yield_all[list_keep]
    #year_all = year_all[list_keep]
    #print(year_all)
    #locations_all = locations_all[list_keep,:]
    #index_all = index_all[list_keep,:]

    # split into train and validate
    print(year_all)
    print(year_all < predict_year)
    index_train = np.nonzero(year_all < predict_year)[0]
    index_validate = np.nonzero(year_all == predict_year)[0]
    print('train size',index_train)
    print('validate size',index_validate)

    # calc train image mean (for each band), and then detract (broadcast)
    image_mean=np.mean(image_all[index_train],(0,1,2))
    image_all = image_all - image_mean
    print(image_all.shape)
    image_validate=image_all[index_validate]
    print(image_validate.shape)
    yield_validate=yield_all[index_validate]


    model= NeuralModel(config,'net')

    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.22)
    # Launch the graph.
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
    sess.run(tf.initialize_all_variables())

    summary_train_loss = []
    summary_eval_loss = []
    summary_RMSE = []
    summary_ME = []

    train_loss=0
    val_loss=0
    val_prediction = 0
    val_deviation = np.zeros([config.B])
    # #########################
    # block when test
    # add saver
    saver=tf.train.Saver()
    # Restore variables from disk.
    try:
        saver.restore(sess, config.save_path+str(predict_year)+"CNN_model.ckpt")
    # Restore log results
        npzfile = np.load(config.save_path + str(predict_year)+'result.npz')
        summary_train_loss = npzfile['summary_train_loss'].tolist()
        summary_eval_loss = npzfile['summary_eval_loss'].tolist()
        summary_RMSE = npzfile['summary_RMSE'].tolist()
        summary_ME = npzfile['summary_ME'].tolist()
        print(("Model restored."))
    except:
        print('No history model found')
    # #########################
    

    RMSE_min = 100
    try:
        for i in range(config.train_step):
            print(i)
            if i==3500:
                config.lr/=10
                # saver.restore(sess, config.save_path+str(predict_year)+"CNN_model.ckpt")
                # # Restore log results
                # npzfile = np.load(config.save_path + str(predict_year)+'result.npz')
                # summary_train_loss = npzfile['summary_train_loss'].tolist()
                # summary_eval_loss = npzfile['summary_eval_loss'].tolist()
                # summary_RMSE = npzfile['summary_RMSE'].tolist()
                # summary_ME = npzfile['summary_ME'].tolist()
                # print("Model restored.")
            if i==20000:
                config.lr/=10
                # saver.restore(sess, config.save_path+str(predict_year)+"CNN_model.ckpt")
                # # Restore log results
                # npzfile = np.load(config.save_path + str(predict_year)+'result.npz')
                # summary_train_loss = npzfile['summary_train_loss'].tolist()
                # summary_eval_loss = npzfile['summary_eval_loss'].tolist()
                # summary_RMSE = npzfile['summary_RMSE'].tolist()
                # summary_ME = npzfile['summary_ME'].tolist()
                # print("Model restored.")
            # if i==12000:
            #     config.lr/=10
                # saver.restore(sess, config.save_path+str(predict_year)+"CNN_model.ckpt")
                # # Restore log results
                # npzfile = np.load(config.save_path + str(predict_year)+'result.npz')
                # summary_train_loss = npzfile['summary_train_loss'].tolist()
                # summary_eval_loss = npzfile['summary_eval_loss'].tolist()
                # summary_RMSE = npzfile['summary_RMSE'].tolist()
                # summary_ME = npzfile['summary_ME'].tolist()
                # print("Model restored.")

            # No augmentation
            # index_train_batch = np.random.choice(index_train,size=config.B)
            # image_train_batch = image_all[index_train_batch,:,0:config.H,:]
            # yield_train_batch = yield_all[index_train_batch]
            # year_train_batch = year_all[index_train_batch,np.newaxis]

            # try data augmentation while training
            index_train_batch_1 = np.random.choice(index_train,size=config.B)
            index_train_batch_2 = np.random.choice(index_train,size=config.B)
            image_train_batch = (image_all[index_train_batch_1,:,0:config.H,:]+image_all[index_train_batch_1,:,0:config.H,:])/2
            yield_train_batch = (yield_all[index_train_batch_1]+yield_all[index_train_batch_1])/2
            # year_train_batch = (year_all[index_train_batch_1,np.newaxis]+year_all[index_train_batch_2,np.newaxis])/2

            index_validate_batch = np.random.choice(index_validate, size=config.B)

            _, train_loss = sess.run([model.train_op, model.loss_err], feed_dict={
                model.x:image_train_batch,
                model.y:yield_train_batch,
                model.lr:config.lr,
                model.keep_prob: config.drop_out
                })

            if i%200 == 0:
                val_loss,fc6,W,B = sess.run([model.loss_err,model.fc6,model.dense_W,model.dense_B], feed_dict={
                    model.x: image_all[index_validate_batch, :, 0:config.H, :],
                    model.y: yield_all[index_validate_batch],
                    model.keep_prob: 1
                })

                print('predict year'+str(predict_year)+'step'+str(i),train_loss,val_loss,config.lr)
                logging.info('predict year %d step %d %f %f %f',predict_year,i,train_loss,val_loss,config.lr)
            if i%200 == 0:
                # do validation
                pred = []
                real = []
                print(image_validate.shape)
                for j in range(int(image_validate.shape[0] / config.B)):
                    real_temp = yield_validate[j * config.B:(j + 1) * config.B]
                    pred_temp= sess.run(model.logits, feed_dict={
                        model.x: image_validate[j * config.B:(j + 1) * config.B,:,0:config.H,:],
                        model.y: yield_validate[j * config.B:(j + 1) * config.B],
                        model.keep_prob: 1
                        })
                    pred.append(pred_temp)
                    real.append(real_temp)
                    print(pred)
                    print(real)
                pred=np.concatenate(pred)
                real=np.concatenate(real)
                RMSE=np.sqrt(np.mean((pred-real)**2))
                ME=np.mean(pred-real)

                if RMSE<RMSE_min:
                    RMSE_min=RMSE
                    # # save
                    # save_path = saver.save(sess, config.save_path + str(predict_year)+'CNN_model.ckpt')
                    # print('save in file: %s' % save_path)
                    # np.savez(config.save_path+str(predict_year)+'result.npz',
                    #     summary_train_loss=summary_train_loss,summary_eval_loss=summary_eval_loss,
                    #     summary_RMSE=summary_RMSE,summary_ME=summary_RMSE)

                print('Validation set','RMSE',RMSE,'ME',ME,'RMSE_min',RMSE_min)
                logging.info('Validation set RMSE %f ME %f RMSE_min %f',RMSE,ME,RMSE_min)
            
                summary_train_loss.append(train_loss)
                summary_eval_loss.append(val_loss)
                summary_RMSE.append(RMSE)
                summary_ME.append(ME)



    except KeyboardInterrupt:
        print('stopped')

    finally:

        # save
        save_path = saver.save(sess, config.save_path + str(predict_year)+'CNN_model.ckpt')
        print('save in file: %s' % save_path)
        logging.info('save in file: %s' % save_path)

        # save result
        pred_out = []
        real_out = []
        feature_out = []
        year_out = []
        locations_out =[]
        index_out = []
        for i in range(int(image_all.shape[0] / config.B)):
            feature,pred = sess.run(
                [model.fc6,model.logits], feed_dict={
                model.x: image_all[i * config.B:(i + 1) * config.B,:,0:config.H,:],
                model.y: yield_all[i * config.B:(i + 1) * config.B],
                model.keep_prob:1
            })
            real = yield_all[i * config.B:(i + 1) * config.B]

            pred_out.append(pred)
            real_out.append(real)
            feature_out.append(feature)
            year_out.append(year_all[i * config.B:(i + 1) * config.B])
            locations_out.append(locations_all[i * config.B:(i + 1) * config.B])
            index_out.append(index_all[i * config.B:(i + 1) * config.B])
            # print i
        weight_out, b_out = sess.run(
            [model.dense_W, model.dense_B], feed_dict={
                model.x: image_all[0 * config.B:(0 + 1) * config.B, :, 0:config.H, :],
                model.y: yield_all[0 * config.B:(0 + 1) * config.B],
                model.keep_prob: 1
            })
        pred_out=np.concatenate(pred_out)
        real_out=np.concatenate(real_out)
        feature_out=np.concatenate(feature_out)
        year_out=np.concatenate(year_out)
        locations_out=np.concatenate(locations_out)
        index_out=np.concatenate(index_out)
        
        path = config.save_path + str(predict_year)+'result_prediction.npz'
        np.savez(path,
            pred_out=pred_out,real_out=real_out,feature_out=feature_out,
            year_out=year_out,locations_out=locations_out,weight_out=weight_out,b_out=b_out,index_out=index_out)

        # RMSE_GP,ME_GP,Average_GP=GaussianProcess(predict_year,path)
        # print 'RMSE_GP',RMSE_GP
        # print 'ME_GP',ME_GP
        # print 'Average_GP',Average_GP

        np.savez(config.save_path+str(predict_year)+'result.npz',
                        summary_train_loss=summary_train_loss,summary_eval_loss=summary_eval_loss,
                        summary_RMSE=summary_RMSE,summary_ME=summary_ME)
        # plot results
        npzfile = np.load(config.save_path+str(predict_year)+'result.npz')
        summary_train_loss=npzfile['summary_train_loss']
        summary_eval_loss=npzfile['summary_eval_loss']
        summary_RMSE = npzfile['summary_RMSE']
        summary_ME = npzfile['summary_ME']

        # Plot the points using matplotlib
        plt.plot(range(len(summary_train_loss)), summary_train_loss)
        plt.plot(range(len(summary_eval_loss)), summary_eval_loss)
        plt.xlabel('Training steps')
        plt.ylabel('L2 loss')
        plt.title('Loss curve')
        plt.legend(['Train', 'Validate'])
        plt.show()

        plt.plot(range(len(summary_RMSE)), summary_RMSE)
        # plt.plot(range(len(summary_ME)), summary_ME)
        plt.xlabel('Training steps')
        plt.ylabel('Error')
        plt.title('RMSE')
        # plt.legend(['RMSE', 'ME'])
        plt.show()

        # plt.plot(range(len(summary_RMSE)), summary_RMSE)
        plt.plot(range(len(summary_ME)), summary_ME)
        plt.xlabel('Training steps')
        plt.ylabel('Error')
        plt.title('ME')
        # plt.legend(['RMSE', 'ME'])
        plt.show()

/home/neelansh/Desktop/Crop/OUTPUT_FULL/
[2016. 2016. 2016. ...    0.    0.    0.]
[2016. 2016. 2016. ... 2003. 2003. 2003.]
[2016. 2016. 2016. ... 2003. 2003. 2003.]
[False False False ...  True  True  True]
train size [ 1363  1364  1365 ... 19218 19219 19220]
validate size [   0    1    2 ... 1360 1361 1362]
(19221, 32, 32, 9)
(1363, 32, 32, 9)
(?, 8192)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
INFO:tensorflow:Restoring parameters from /home/neelansh/Desktop/Crop/TRAIN_RESULTS/2016CNN_model.ckpt
Model restored.
0
predict year2016step0 16144.904 11487.973 0.001
(1363, 32, 32, 9)
[array([149.8868  ,  82.91738 , 143.0436  , 193.06479 , 134.4319  ,
       165.39839 , 194.81593 , 126.173615, 131.30118 , 146.40952 ,
       109.165054, 123.9276  , 101.061066, 130.59853 , 160.69374 ,
       147.56375 , 123.13228 , 129.06673 , 145.40698 , 133.48566 ,
       112.92348 , 139.86887 ,  88.350174, 103.49364 , 104.870186,
       129.30168 , 149.20891 , 124.24983 , 1

[array([149.8868  ,  82.91738 , 143.0436  , 193.06479 , 134.4319  ,
       165.39839 , 194.81593 , 126.173615, 131.30118 , 146.40952 ,
       109.165054, 123.9276  , 101.061066, 130.59853 , 160.69374 ,
       147.56375 , 123.13228 , 129.06673 , 145.40698 , 133.48566 ,
       112.92348 , 139.86887 ,  88.350174, 103.49364 , 104.870186,
       129.30168 , 149.20891 , 124.24983 , 190.59682 , 174.23433 ,
       107.19516 , 224.63393 ], dtype=float32), array([173.66002 , 166.4199  , 161.06306 , 162.18217 , 170.81676 ,
       163.08977 , 153.29459 , 174.0863  , 136.03917 , 141.77736 ,
       147.31898 , 158.30553 , 172.69044 , 127.61966 , 157.9728  ,
       165.37218 , 174.66164 , 174.20932 , 148.56062 , 167.93623 ,
       100.86899 , 111.517845, 102.647125, 125.74242 , 121.86488 ,
       116.57679 ,  72.05006 ,  86.73967 ,  69.88644 , 109.283325,
        74.888214,  94.63011 ], dtype=float32), array([162.5905  , 142.0343  ,  65.27744 ,  63.49162 ,  90.49832 ,
        66.46065 ,  90.21402 , 1

[array([149.8868  ,  82.91738 , 143.0436  , 193.06479 , 134.4319  ,
       165.39839 , 194.81593 , 126.173615, 131.30118 , 146.40952 ,
       109.165054, 123.9276  , 101.061066, 130.59853 , 160.69374 ,
       147.56375 , 123.13228 , 129.06673 , 145.40698 , 133.48566 ,
       112.92348 , 139.86887 ,  88.350174, 103.49364 , 104.870186,
       129.30168 , 149.20891 , 124.24983 , 190.59682 , 174.23433 ,
       107.19516 , 224.63393 ], dtype=float32), array([173.66002 , 166.4199  , 161.06306 , 162.18217 , 170.81676 ,
       163.08977 , 153.29459 , 174.0863  , 136.03917 , 141.77736 ,
       147.31898 , 158.30553 , 172.69044 , 127.61966 , 157.9728  ,
       165.37218 , 174.66164 , 174.20932 , 148.56062 , 167.93623 ,
       100.86899 , 111.517845, 102.647125, 125.74242 , 121.86488 ,
       116.57679 ,  72.05006 ,  86.73967 ,  69.88644 , 109.283325,
        74.888214,  94.63011 ], dtype=float32), array([162.5905  , 142.0343  ,  65.27744 ,  63.49162 ,  90.49832 ,
        66.46065 ,  90.21402 , 1

[array([149.8868  ,  82.91738 , 143.0436  , 193.06479 , 134.4319  ,
       165.39839 , 194.81593 , 126.173615, 131.30118 , 146.40952 ,
       109.165054, 123.9276  , 101.061066, 130.59853 , 160.69374 ,
       147.56375 , 123.13228 , 129.06673 , 145.40698 , 133.48566 ,
       112.92348 , 139.86887 ,  88.350174, 103.49364 , 104.870186,
       129.30168 , 149.20891 , 124.24983 , 190.59682 , 174.23433 ,
       107.19516 , 224.63393 ], dtype=float32), array([173.66002 , 166.4199  , 161.06306 , 162.18217 , 170.81676 ,
       163.08977 , 153.29459 , 174.0863  , 136.03917 , 141.77736 ,
       147.31898 , 158.30553 , 172.69044 , 127.61966 , 157.9728  ,
       165.37218 , 174.66164 , 174.20932 , 148.56062 , 167.93623 ,
       100.86899 , 111.517845, 102.647125, 125.74242 , 121.86488 ,
       116.57679 ,  72.05006 ,  86.73967 ,  69.88644 , 109.283325,
        74.888214,  94.63011 ], dtype=float32), array([162.5905  , 142.0343  ,  65.27744 ,  63.49162 ,  90.49832 ,
        66.46065 ,  90.21402 , 1

[array([149.8868  ,  82.91738 , 143.0436  , 193.06479 , 134.4319  ,
       165.39839 , 194.81593 , 126.173615, 131.30118 , 146.40952 ,
       109.165054, 123.9276  , 101.061066, 130.59853 , 160.69374 ,
       147.56375 , 123.13228 , 129.06673 , 145.40698 , 133.48566 ,
       112.92348 , 139.86887 ,  88.350174, 103.49364 , 104.870186,
       129.30168 , 149.20891 , 124.24983 , 190.59682 , 174.23433 ,
       107.19516 , 224.63393 ], dtype=float32), array([173.66002 , 166.4199  , 161.06306 , 162.18217 , 170.81676 ,
       163.08977 , 153.29459 , 174.0863  , 136.03917 , 141.77736 ,
       147.31898 , 158.30553 , 172.69044 , 127.61966 , 157.9728  ,
       165.37218 , 174.66164 , 174.20932 , 148.56062 , 167.93623 ,
       100.86899 , 111.517845, 102.647125, 125.74242 , 121.86488 ,
       116.57679 ,  72.05006 ,  86.73967 ,  69.88644 , 109.283325,
        74.888214,  94.63011 ], dtype=float32), array([162.5905  , 142.0343  ,  65.27744 ,  63.49162 ,  90.49832 ,
        66.46065 ,  90.21402 , 1

...

[array([143.62141 ,  83.23554 , 138.8116  , 188.3382  , 129.17696 ,
       162.9459  , 193.03246 , 125.85771 , 129.90007 , 142.94583 ,
       112.59119 , 123.26877 , 103.476006, 133.12831 , 151.80733 ,
       142.07771 , 121.81523 , 122.89421 , 138.07793 , 128.12357 ,
       113.71208 , 132.1909  ,  93.70527 , 111.09032 , 104.449615,
       130.06773 , 145.40208 , 125.83917 , 185.37721 , 168.4432  ,
       111.14351 , 224.76393 ], dtype=float32), array([167.6649  , 159.42589 , 153.49571 , 154.67677 , 165.47752 ,
       156.1169  , 144.55977 , 167.05591 , 129.15007 , 138.0453  ,
       139.29128 , 148.21938 , 164.81607 , 118.49736 , 151.1941  ,
       156.56308 , 167.49232 , 168.56934 , 143.43831 , 157.97275 ,
       102.63406 , 115.1418  , 103.2792  , 120.27446 , 112.047806,
       112.50193 ,  68.777084,  84.76988 ,  68.68628 , 104.37799 ,
        70.064835,  91.86763 ], dtype=float32), array([153.74333 , 136.56758 ,  67.51891 ,  61.978577,  91.31455 ,
        67.02028 ,  92.41637 , 1

[array([143.62141 ,  83.23554 , 138.8116  , 188.3382  , 129.17696 ,
       162.9459  , 193.03246 , 125.85771 , 129.90007 , 142.94583 ,
       112.59119 , 123.26877 , 103.476006, 133.12831 , 151.80733 ,
       142.07771 , 121.81523 , 122.89421 , 138.07793 , 128.12357 ,
       113.71208 , 132.1909  ,  93.70527 , 111.09032 , 104.449615,
       130.06773 , 145.40208 , 125.83917 , 185.37721 , 168.4432  ,
       111.14351 , 224.76393 ], dtype=float32), array([167.6649  , 159.42589 , 153.49571 , 154.67677 , 165.47752 ,
       156.1169  , 144.55977 , 167.05591 , 129.15007 , 138.0453  ,
       139.29128 , 148.21938 , 164.81607 , 118.49736 , 151.1941  ,
       156.56308 , 167.49232 , 168.56934 , 143.43831 , 157.97275 ,
       102.63406 , 115.1418  , 103.2792  , 120.27446 , 112.047806,
       112.50193 ,  68.777084,  84.76988 ,  68.68628 , 104.37799 ,
        70.064835,  91.86763 ], dtype=float32), array([153.74333 , 136.56758 ,  67.51891 ,  61.978577,  91.31455 ,
        67.02028 ,  92.41637 , 1

[array([143.62141 ,  83.23554 , 138.8116  , 188.3382  , 129.17696 ,
       162.9459  , 193.03246 , 125.85771 , 129.90007 , 142.94583 ,
       112.59119 , 123.26877 , 103.476006, 133.12831 , 151.80733 ,
       142.07771 , 121.81523 , 122.89421 , 138.07793 , 128.12357 ,
       113.71208 , 132.1909  ,  93.70527 , 111.09032 , 104.449615,
       130.06773 , 145.40208 , 125.83917 , 185.37721 , 168.4432  ,
       111.14351 , 224.76393 ], dtype=float32), array([167.6649  , 159.42589 , 153.49571 , 154.67677 , 165.47752 ,
       156.1169  , 144.55977 , 167.05591 , 129.15007 , 138.0453  ,
       139.29128 , 148.21938 , 164.81607 , 118.49736 , 151.1941  ,
       156.56308 , 167.49232 , 168.56934 , 143.43831 , 157.97275 ,
       102.63406 , 115.1418  , 103.2792  , 120.27446 , 112.047806,
       112.50193 ,  68.777084,  84.76988 ,  68.68628 , 104.37799 ,
        70.064835,  91.86763 ], dtype=float32), array([153.74333 , 136.56758 ,  67.51891 ,  61.978577,  91.31455 ,
        67.02028 ,  92.41637 , 1

stopped
save in file: /home/neelansh/Desktop/Crop/TRAIN_RESULTS/2016CNN_model.ckpt


KeyboardInterrupt: 